In [16]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import pandas as pd

load_dotenv()


True

In [19]:
MODEL = "gpt-3.5-turbo"

OUTPUT_FOLDER = "../generated_GUIs/"

IMAGE_INPUT_FOLDER = "../data/"

CREATION_PROMPT = PromptTemplate(
    input_variables=["summary"],
    template= """create the HTML code for a Mobilepage, 
    with a bit of styling, that represents following functionality: 
    {summary}."""
)

SUMMARY_PROMPT = PromptTemplate(
    input_variables=["sum1","sum2","sum3","sum4","sum5"],
    template=
        """Given a collection of Screensummaries, which descripe the same screen:
        Summary 1: {sum1}.
        Summary 2: {sum2}.
        Summary 3: {sum3}.
        Summary 4: {sum4}.
        Summary 5: {sum5}.

        Extract the provided information of the screen and formulate a new description."""
)

S2W = pd.read_csv("../data/Screen2Words.csv")
S2W = S2W.drop('Unnamed: 0', axis=1)

In [3]:
#function for Creating a GUI based on a given description
def create_gui(id, summary, prompt=CREATION_PROMPT, temp=0):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp)

    chain = LLMChain(llm=llm, prompt=prompt)

    try: 
        response = chain.run(summary=summary)
    except Exception as e:
        print(e)
    
    print(response)
    with open(OUTPUT_FOLDER + f"{id}.html", 'w') as f:
        f.write(response)


In [4]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser

#Sequential chain, of summarizing the Screen2Words Summaries into one and creating a GUI
def create_gui_S2W(id, sum1, sum2, sum3, sum4, sum5, temp=0.5):
    llm = ChatOpenAI(model_name=MODEL, temperature=temp)
    
    summary_chain = SUMMARY_PROMPT | llm | StrOutputParser()
    creation_chain = CREATION_PROMPT | llm | StrOutputParser()
    
    chain = {"summary": summary_chain} | RunnablePassthrough.assign(creation=creation_chain)
    
    try: 
        response = chain.invoke({"sum1": sum1,"sum2": sum2,"sum3": sum3,"sum4": sum4,"sum5": sum5})
    except Exception as e:
        print(e)
    
    print(response)
    with open(OUTPUT_FOLDER + f"{id}_S2W.html", 'w') as f:
        f.write(response.get('creation'))
    return response

In [5]:
import os
import base64
import requests

#decode screenshot into 64 encoded images
def encode_image(id):
    image_path = IMAGE_INPUT_FOLDER + f"{id}.jpg"
    print(image_path)
    with open(image_path,"rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')    
    
def describe_image(id):
    api_key = os.getenv("OPENAI_API_KEY")
    base64_image = encode_image(id)

    print(base64_image)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": "gpt-4-vision-preview",
        "messages": [
        {
            "role": "user",
            "content": [
            {
                "type": "text",
                "text": "given a screen, summarize its purpose briefly"
            },
            {
                "type": "image_url",
                "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
                }
            }
            ]
        }
        ],
        "max_tokens": 300
    }
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return response

In [6]:
description = describe_image(455) 

../data/455.jpg
...


In [29]:
description.json().get("choices")[0].get("message").get("content")

'This screen appears to be part of a public transportation or bus route app or website. Its purpose is to display a specific bus route along with the stops it makes. Users can likely select different routes, view schedules for weekdays (Wkd), Saturdays (Sat), and Sundays (Sun), and access additional resources like a printable PDF of the route map or a turn-by-turn text description of the route. The map shown lists stops sequentially for a bus heading Northbound, suggesting that this is a detailed view for passengers planning their travel in that direction on the 1 South Congress/Metric route during the weekdays.'